In [1]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
import numpy as np
from keras.utils import np_utils
import tensorflow as tf
from keras import optimizers

Using TensorFlow backend.


In [2]:
from keras import backend as K

def mcor(y_true, y_pred):
    #matthews_correlation
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos


    y_pos = K.round(K.clip(y_true, 0, 1))
    y_neg = 1 - y_pos


    tp = K.sum(y_pos * y_pred_pos)
    tn = K.sum(y_neg * y_pred_neg)


    fp = K.sum(y_neg * y_pred_pos)
    fn = K.sum(y_pos * y_pred_neg)


    numerator = (tp * tn - fp * fn)
    denominator = K.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return numerator / (denominator + K.epsilon())

def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

In [3]:
# X has shape (num_rows, num_cols), where the training data are stored as row vectors
# X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=np.float32)
# y must have an output vector for each input vector
# y = np.array([[0], [0], [0], [1]], dtype=np.float32)
# Set random seed
np.random.seed(42)

X = np.array([[0,0],[0,1],[1,0],[1,1]]).astype('float32')
y = np.array([[0],[1],[1],[0]]).astype('float32')

# One-hot encoding the output
y = np_utils.to_categorical(y)

In [13]:
# Create the Sequential model
model = Sequential()
# 1st Layer - Add an input layer of 32 nodes with the same input shape as
# the training samples in X
# Keras requires the input shape to be specified in the first layer
model.add(Dense(32, input_dim=2))
# 2nd Layer - Add a fully connected output layer
model.add(Activation("sigmoid"))
model.add(Dense(2))
# Add a sigmoid activation layer
model.add(Activation('sigmoid'))

# teste = Sequential()
# teste.add(Dense(32, input_dim=2))
# teste.add(Activation("sigmoid"))
# teste.add(Dense(2))
# teste.add(Activation("sigmoid"))

In [179]:
sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
RMSprop = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
Adagrad = optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
Adadelta = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
Adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
Adamax = optimizers.Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
Nadam = optimizers.Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)

# model.compile(loss='mean_squared_error', optimizer=sgd)
# model.compile(loss='mean_squared_error', optimizer='sgd')

# binary_crossentropy      -> 0.88888895511627197
# categorical_crossentropy -> 0.88888895511627197
# mean_squared_error       -> 0.88888895511627197

model.compile(loss='mean_squared_error', optimizer= "adam", metrics=[mcor, recall, f1, 'accuracy'])
# model.compile(loss='binary_crossentropy', optimizer=RMSprop, metrics=[mcor,recall, f1, 'accuracy'])
# model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=[mcor,recall, f1, 'accuracy'])
# model.compile(loss='binary_crossentropy', optimizer=Adagrad, metrics=[mcor,recall, f1, 'accuracy'])
# model.compile(loss='binary_crossentropy', optimizer=Adadelta, metrics=[mcor,recall, f1, 'accuracy'])
# model.compile(loss='binary_crossentropy', optimizer=Adam, metrics=[mcor,recall, f1, 'accuracy'])
# model.compile(loss='binary_crossentropy', optimizer=Adamax, metrics=[mcor,recall, f1, 'accuracy'])
# model.compile(loss='binary_crossentropy', optimizer=Nadam, metrics=[mcor,recall, f1, 'accuracy'])
# model.compile(loss="categorical_crossentropy", optimizer="adam", metrics = ['accuracy'])
# model.compile(loss="mean_squared_error", optimizer="adam", metrics = ['accuracy'])
# model.compile(loss="mean_squared_error", optimizer=RMSprop, metrics = ['accuracy'])
# model.compile(loss="mean_squared_error", optimizer=sgd, metrics = ['accuracy'])

In [180]:
# model.summary()

In [181]:
model.fit(X, y, epochs=1000, verbose=0)

In [182]:
# Scoring the model
score = model.evaluate(X, y)
print("Accuracy: ", score[-1])
print("Loss: ", score[0])
print("mcor: ", score[1])
print("recall: ", score[2])
print("f1: ", score[3])

# Checking the predictions
print("\nPredictions:")
print(model.predict_proba(X))

4/4 [==============================] - 1s 193ms/step
('Accuracy: ', 0.75)
('Loss: ', 0.125)
('mcor: ', 0.77459663152694702)
('recall: ', 1.0)
('f1: ', 0.88888895511627197)

Predictions:
[[  9.99999881e-01   7.62263355e-08]
 [  1.00000000e+00   1.00000000e+00]
 [  8.89055656e-08   9.99999881e-01]
 [  1.00000000e+00   5.92950862e-08]]
